In [125]:
import sqlite3 as db
import pandas as pd
import featuretools as ft
import json

In [126]:
# Hacer una query SQL
def sql_query(q):
    conn = db.connect('../db/sqlite/eicu_v2_0_1.sqlite3')
    df = pd.read_sql_query(q, conn)
    conn.close()
    
    return df

# Leer todos los CSV
def read_csvs():
    #import os
    #print( os.getcwd())
    datasets = [ 'admissiondrug', 'admissionDx', 'allergy', 'apacheApsVar', 'apachePatientResult', 'apachePredVar', 'carePlanCareProvider', 'carePlanEOL', 'carePlanGeneral',
                 'carePlanGoal', 'carePlanInfectiousDisease', 'customLab', 'diagnosis', 'hospital', 'infusiondrug', 'intakeOutput', 'lab', 'medication', 'microLab', 'note',
                 'nurseAssessment', 'nurseCare', 'nurseCharting', 'pastHistory', 'patient', 'physicalExam', 'respiratoryCare', 'respiratoryCharting', 'treatment', 'vitalAperiodic',
                 'vitalPeriodic']

    dfs = {}

    for ds_name in datasets:
        dfs[ds_name] = pd.read_csv('../db/csv_clean/' + ds_name + '.csv')
    
    return dfs

dfs = read_csvs()
has_dropped_keys = False # Para que no se droppeen cada vez que se ejecuta la celda siguiente

In [127]:
dfs['diagnosis'] = dfs['diagnosis'].drop_duplicates()
dfs['diagnosis'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24748 entries, 0 to 24747
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   patientunitstayid    24748 non-null  int64 
 1   activeupondischarge  24748 non-null  bool  
 2   diagnosisoffset      24748 non-null  int64 
 3   diagnosisstring      24748 non-null  object
 4   icd9code             21012 non-null  object
 5   diagnosispriority    24748 non-null  object
dtypes: bool(1), int64(2), object(3)
memory usage: 1.2+ MB


In [128]:
# Crear _last_diagnosis a partir de diagnosisstring

def clean_diagnosisstring(row):
    if row['diagnosisstring'] != row['diagnosisstring']: # If NaN
        return ''
    else:
        return row['diagnosisstring'].split('|')[-1] + ' / ' + row['diagnosispriority']
    
dfs['diagnosis']['_all_diagnosis'] = dfs['diagnosis'].apply(clean_diagnosisstring, axis=1)

In [129]:
dfs['diagnosis'] = dfs['diagnosis'].drop(columns=[
    'activeupondischarge',
    'diagnosisstring',
    'icd9code',
    'diagnosispriority'
]) # No creo que importe si el análisis se ha resuelto en la estancia

In [137]:
dfs['diagnosis']['diagnosisoffset'] = dfs['diagnosis'].apply(lambda row: (int(row['diagnosisoffset'])//50) * 50, axis=1)
dfs['diagnosis'] = dfs['diagnosis'].drop_duplicates()

In [138]:
#for i, row in dfs['diagnosis'].iterrows():
#    print(row['icd9code'], ' -> ', row['diagnosisstring'])

pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 400)
dfs['diagnosis'].head(400)

,patientunitstayid,diagnosisoffset,_all_diagnosis
0,346380,5000,hypertension / Other
1,346380,650,change in mental status / Major
2,346380,5000,hypotension / Major
3,346380,5000,schizophrenia / Major
4,346380,5000,thrombus / Major
6,346380,5000,acute renal failure / Primary
7,346380,5000,syncope / Major
8,346380,5000,Parkinson's disease / Major
9,346380,5000,dehydration / Major
10,346380,5000,COPD / Major
